# Analyzing frequency and power spectrum of significant cells.
* using raw fluorescence data 
* 11/12/2023 fixing a bug when filtering significant and non significant cells

In [ ]:
import sys
from os.path import join
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

sys.path.append("/home/gergely/code/sleep/src/")

from classes.suite2p_class import Suite2p
from classes.eeg_class import EegData
from classes.behavior_class import BehaviorData
import neuropil_io as nl

In [ ]:
sima_folders = [
    "/data2/gergely/invivo_DATA/sleep/5HT2afl05b_1/7_7_SD/TSeries-07072021-0833_GC1-001/TSeries-07072021-0833_GC1-001_Cycle00001_Element00001.sima",
    "/data2/gergely/invivo_DATA/sleep/5HT2afl05b_1/7_7_SD/TSeries-07072021-0833_GC2-003/TSeries-07072021-0833_GC2-003_Cycle00001_Element00001.sima",
    "/data2/gergely/invivo_DATA/sleep/5HT2afl05b_1/7_7_SD/TSeries-07072021-0833_MC1-002/TSeries-07072021-0833_MC1-002_Cycle00001_Element00001.sima",
    "/data2/gergely/invivo_DATA/sleep/5HT2afl05b_1/7_7_SD/TSeries-07072021-0833_MC2-004/TSeries-07072021-0833_MC2-004_Cycle00001_Element00001.sima",
    "/data2/gergely/invivo_DATA/sleep/5HT2afl05b_2/7_6_SD/TSeries-07062021-0844_GC-001/TSeries-07062021-0844_GC-001_Cycle00001_Element00001.sima",
    "/data2/gergely/invivo_DATA/sleep/5HT2afl05b_2/7_6_SD/TSeries-07062021-0844_MC-002/TSeries-07062021-0844_MC-002_Cycle00001_Element00001.sima",
    "/data2/gergely/invivo_DATA/sleep/5HT2afl05b_2/7_6_SD/TSeries-07062021-0844_MC2-004/TSeries-07062021-0844_MC2-004_Cycle00001_Element00001.sima",
    "/data2/gergely/invivo_DATA/sleep/sert52b_5/4_3_SD/TSeries-04032023-1323-001/TSeries-04032023-1323-001.sima",
    "/data2/gergely/invivo_DATA/sleep/sert54a_5/4_4/TSeries-04042023-1023_sess1-001/TSeries-04042023-1023_sess1-001.sima",
    "/data2/gergely/invivo_DATA/sleep/sert52b_1/4_3_SD/TSeries-04032023-1323_SD_sess1-001/TSeries-04032023-1323_SD_sess1-001.sima",
]

In [ ]:
results = {}

for sima_folder in sima_folders[1:]:
    print(sima_folder)
    sima_dict = {}

    # Load eeg data
    file_name = join(sima_folder, "eeg", "velo_eeg.csv")
    eeg_velo = EegData.load_load_processed_velocity_eeg(file_name=file_name)
    eeg_velo["mobile_immobile"] = BehaviorData.define_mobility(
        eeg_velo["filtered velo"]
    )
    df = EegData.brain_state_filter(
        velo_eeg_df=eeg_velo,
        states=["awake_mobile", "awake_immobile", "NREM", "REM", "other"],
    )

    s2p_folder = join(sima_folder, "suite2p")
    s2p_data = Suite2p(s2p_folder=s2p_folder)
    cells = s2p_data.cells()
    npil = s2p_data.npil()

    nrem_cells = cells[:, df["NREM"].values]
    awake_immobile_cells = cells[:, df["awake_immobile"].values]

    significant_cells = nl.statistics.significance_calc(
        nrem_cells, awake_immobile_cells
    )
    # creating mask for significant cell
    mask = np.zeros(cells.shape[0], dtype=bool)

    mask[significant_cells] = True

    significant_cell_data = cells[mask, :]
    nonsignificant_cell_data = cells[~mask, :]

    mean_significant = np.mean(significant_cell_data, axis=0)
    mean_nonsignificant = np.mean(nonsignificant_cell_data, axis=0)

    nrem_sig_mean = pd.Series(mean_significant[df["NREM"].values])
    awake_sig_mean = pd.Series(mean_significant[df["awake_immobile"].values])
    nrem_nonsig_mean = pd.Series(mean_nonsignificant[df["NREM"].values])
    awake_nonsig_mean = pd.Series(mean_nonsignificant[df["awake_immobile"].values])

    nrem_sig_freq, nrem_sig_psd = nl.freq_calc(nrem_sig_mean, resolution=0.01)
    awake_sig_freq, awake_sig_psd = nl.freq_calc(awake_sig_mean, resolution=0.01)
    nrem_nonsig_freq, nrem_nonsig_psd = nl.freq_calc(nrem_nonsig_mean, resolution=0.01)
    awake_nonsig_freq, awake_nonsig_psd = nl.freq_calc(
        awake_nonsig_mean, resolution=0.01
    )

    sima_dict["awake_sig"] = {
        "freq": awake_sig_freq,
        "psd": awake_sig_psd,
    }

    sima_dict["nrem_sig"] = {
        "freq": nrem_sig_freq,
        "psd": nrem_sig_psd,
    }

    sima_dict["awake_nonsig"] = {
        "freq": awake_nonsig_freq,
        "psd": awake_nonsig_psd,
    }

    sima_dict["nrem_nonsig"] = {
        "freq": nrem_nonsig_freq,
        "psd": nrem_nonsig_psd,
    }

    results[sima_folder] = sima_dict

In [ ]:
results_df = pd.DataFrame.from_dict(results)
results_df

In [ ]:
# 1. Extract PSD arrays
psd_df = results_df.applymap(
    lambda d: d["psd"] if isinstance(d, dict) and "psd" in d else np.nan
)
psd_df

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 5), sharey=True)

nl.spectral_density_plot(
    psd_df, ["awake_sig", "nrem_sig"], ax=ax[0], labels={"title": "Significant cells"}
)
nl.spectral_density_plot(
    psd_df,
    ["awake_nonsig", "nrem_nonsig"],
    ax=ax[1],
    labels={"title": "Non-significant cells"},
)

## Neuropil signal

In [ ]:
npil_results = {}

for sima_folder in sima_folders[1:]:
    print(sima_folder)
    sima_dict = {}

    # Load eeg data
    file_name = join(sima_folder, "eeg", "velo_eeg.csv")
    eeg_velo = EegData.load_load_processed_velocity_eeg(file_name=file_name)
    eeg_velo["mobile_immobile"] = BehaviorData.define_mobility(
        eeg_velo["filtered velo"]
    )
    df = eegData.brain_state_filter(
        velo_eeg_df=eeg_velo,
        states=["awake_mobile", "awake_immobile", "NREM", "REM", "other"],
    )

    s2p_folder = join(sima_folder, "suite2p")
    s2p_data = Suite2p(s2p_folder=s2p_folder)
    cells = s2p_data.cells()
    npil = s2p_data.npil()

    nrem_cells = cells[:, df["NREM"].values]
    awake_immobile_cells = cells[:, df["awake_immobile"].values]

    significant_cells = nl.statistics.significance_calc(
        nrem_cells, awake_immobile_cells
    )

    mask = np.zeros(npil.shape[0], dtype=bool)

    mask[significant_cells] = True

    significant_npil = npil[mask, :]
    nonsignificant_npil = npil[~mask, :]

    mean_significant = np.mean(significant_npil, axis=0)
    mean_nonsignificant = np.mean(nonsignificant_npil, axis=0)

    nrem_sig_mean = pd.Series(mean_significant[df["NREM"].values])
    awake_sig_mean = pd.Series(mean_significant[df["awake_immobile"].values])
    nrem_nonsig_mean = pd.Series(mean_nonsignificant[df["NREM"].values])
    awake_nonsig_mean = pd.Series(mean_nonsignificant[df["awake_immobile"].values])

    nrem_sig_freq, nrem_sig_psd = nl.freq_calc(nrem_sig_mean, resolution=0.01)
    awake_sig_freq, awake_sig_psd = nl.freq_calc(awake_sig_mean, resolution=0.01)
    nrem_nonsig_freq, nrem_nonsig_psd = nl.freq_calc(nrem_nonsig_mean, resolution=0.01)
    awake_nonsig_freq, awake_nonsig_psd = nl.freq_calc(
        awake_nonsig_mean, resolution=0.01
    )

    sima_dict["awake_sig"] = {
        "freq": awake_sig_freq,
        "psd": awake_sig_psd,
    }

    sima_dict["nrem_sig"] = {
        "freq": nrem_sig_freq,
        "psd": nrem_sig_psd,
    }

    sima_dict["awake_nonsig"] = {
        "freq": awake_nonsig_freq,
        "psd": awake_nonsig_psd,
    }

    sima_dict["nrem_nonsig"] = {
        "freq": nrem_nonsig_freq,
        "psd": nrem_nonsig_psd,
    }

    npil_results[sima_folder] = sima_dict

In [ ]:
npil_results_df = pd.DataFrame.from_dict(npil_results)
npil_results_df

In [ ]:
# 1. Extract PSD arrays
npil_psd_df = npil_results_df.applymap(
    lambda d: d["psd"] if isinstance(d, dict) and "psd" in d else np.nan
)
npil_psd_df

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 5), sharey=True)

nl.spectral_density_plot(
    npil_psd_df,
    ["awake_sig", "nrem_sig"],
    ax=ax[0],
    labels={"title": "Significant cells' npil"},
)
nl.spectral_density_plot(
    npil_psd_df,
    ["awake_nonsig", "nrem_nonsig"],
    ax=ax[1],
    labels={"title": "Non-significant cells' npil"},
)